<a href="https://colab.research.google.com/github/TanmayKumar-EngStud/DeepLearning_Basic-programs/blob/main/LoadingDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Python code for loading and refining Data

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

DataDir = "/content/drive/MyDrive/newSET"
Category = ["women","men"]
print('done')

done


In [3]:
trainingData = []
IMG_SIZE = 250
def create_Training_Data():
    for cat in Category:
        path = os.path.join(DataDir, cat)
        class_num= Category.index(cat)
        for img in os.listdir(path):
            try:
                img_arr   = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
                trainingData.append([new_array, class_num])
            except Exception as e:
                pass
create_Training_Data()            
print(len(trainingData))


3262


In [4]:
import random 
random.shuffle(trainingData)

X= []
y= []
for features, label in trainingData:
    X.append(features)
    y.append(label)


X = np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,1)
print('done')

done


In [5]:
import pickle 

pickle_out = open("X.pickle","wb")
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

X = X/255.0
print('done')

done


## **Creating the model** 


```
# This the actual code of my Convolution network
```


---





In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

## Once Again calling the X and y value to make it a numpy array

In [7]:
X = np.array(X)
y =np.array(y)
print(X.shape[1:])

(250, 250, 1)


## Building the model
---


> Defining the function which creates model




In [8]:
def createModel(DenseLayer, convLayer,LayerSize):

    model= Sequential()
    model.add(Conv2D(LayerSize,(3,3),input_shape= (250,250,1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size= (2,2)))
    
    for i in range (convLayer -1):
      model.add(Conv2D(LayerSize,(3,3)))
      model.add(Activation('relu'))
      model.add(MaxPooling2D(pool_size= (2,2)))


    model.add(Flatten())
    for i in range (DenseLayer-1):
      model.add(Dense(64))
      model.add(Activation('relu'))


    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

# I have compared the model designs with one another
---


*   **Conisdered 4 sets of Dense Layers** (1,2,3,4) 
*   **Considered 3 sets of Neurons in Each layer**  (32,64,128)
*   **Considered 3 sets of convolutional layers**  (2,3,4) 
***
Overall 4x3x3 = 36 Models will be made


In [19]:

!mkdir -p /content/drive/MyDrive/newSET/saved_model

In [20]:
denseLayers = [3,4]
layerSizes = [32, 64]
convLayers = [2,3]

for denseLayer in denseLayers:
  for layerSize in layerSizes:
    for convLayer in convLayers:
      NAME = 'men-x-women-CNN-{}x2-{} -convLayers-{} -denseLayer-{}'.format(layerSize,int(time.time()),convLayer, denseLayer)
      tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/newSET/logs/{}'.format(NAME))
      model = createModel(denseLayer, convLayer,layerSize)
      model.compile(loss ='binary_crossentropy', optimizer ='adam', metrics = ['accuracy'])

      checkpoint_dr = os.path.dirname(checkpoint_path)
      cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only= False, verbose =1)

      model.fit(X,y, batch_size = 16, validation_split=0.1, epochs = 5, callbacks = [tensorboard])
      model.save('/content/drive/MyDrive/newSET/saved_model/training_{}x2 convLayers-{} -denseLayer-{} time-{}'.format(layerSize,convLayer, denseLayer,int(time.time())))



Epoch 1/5
184/184 [==============================] - 5s 26ms/step - loss: 0.7761 - accuracy: 0.5532 - val_loss: 0.6651 - val_accuracy: 0.6086
Epoch 2/5
184/184 [==============================] - 4s 22ms/step - loss: 0.6685 - accuracy: 0.5954 - val_loss: 0.6426 - val_accuracy: 0.6300
Epoch 3/5
184/184 [==============================] - 4s 22ms/step - loss: 0.5635 - accuracy: 0.7231 - val_loss: 0.6784 - val_accuracy: 0.6330
Epoch 4/5
184/184 [==============================] - 4s 22ms/step - loss: 0.3237 - accuracy: 0.8517 - val_loss: 0.8707 - val_accuracy: 0.6086
Epoch 5/5
184/184 [==============================] - 4s 22ms/step - loss: 0.1598 - accuracy: 0.9402 - val_loss: 1.5182 - val_accuracy: 0.6422
INFO:tensorflow:Assets written to: /content/drive/MyDrive/newSET/saved_model/training_32x2 convLayers-2 -denseLayer-3 time-1618132171/assets
Epoch 1/5
184/184 [==============================] - 6s 27ms/step - loss: 0.6900 - accuracy: 0.5545 - val_loss: 0.6841 - val_accuracy: 0.5657
Epoch 2

In [21]:
!tensorboard --logdir /contents/drive/MyDrive/newSET/logs

2021-04-11 09:18:29.729893: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.4.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


#**Saving the model**


---
by using the technique of checkpoint callback 



## After Comparision we can clearly say that:-
>'men-x-women-CNN-32x2-1618119216-convLayers-2-denselayer-3'

is the best as compared to others

Now we will use this Model to further validate our dataset

In [25]:
model = tf.keras.models.load_model('/contents/drive/MyDrive/newSET/saved_model/training_32x2 convLayers-2 -denseLayer-3 time-1618132171/saved_model.pb')

model.fit(X,y, batch_size = 16, validation_split=0.1, epochs =1, callbacks=cp_callback)


OSError: ignored

In [ ]:
!ls {checkpoint_dr}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


# Reloading the previous model
---
Using that model and testing

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/newSET/logs/BestModel.model')

print('done')


OSError: ignored

In [ ]:
import cv2
import tensorflow as tf


Categories = ["women","men"]
def prepare(filepath):
  IMG_SIZE = 250
  img_arr   = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
  new_image = cv2.resize(img_arr, (IMG_SIZE,IMG_SIZE))
  #return img_arr.shape
  return new_image.reshape(-1,IMG_SIZE,IMG_SIZE,1)

prediction = model.predict(prepare("/content/drive/MyDrive/newSET/data/women/02 (6).jpg"))
#prediction = prepare("/content/drive/MyDrive/newSET/data/men/00001713.jpeg")

print(prediction)

[[0.]]
